In [51]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import operator
from bert_embedding import BertEmbedding

from nltk.corpus import stopwords
stop=[]
stop.extend(stopwords.words('english'))

bert_embedding = BertEmbedding()

def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(str.maketrans('', '', string.punctuation))
    filter_str = final_str.translate(str.maketrans('', '', digits))
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stop:
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()


def get_vector(bert_embedding,sentences):
    #sentences 
    result = bert_embedding(sentences,'sum')
    sen_vec = []
    for single_sen,all_word_vec in result:
        comlete_sen_vector = np.array(all_word_vec).mean(axis =0)
        sen_vec.append(comlete_sen_vector)
        
    return sen_vec
#vocab = model.keys()

df_test = pd.read_csv('test_dataset.csv')
df_test.head()
df_train = pd.read_csv('train_dataset.csv')

df_train['source_'] = 1
df_test['source_'] =0

df = pd.concat([df_train,df_test])
df['score_1'].fillna(1, inplace = True)
df['score_2'].fillna(1, inplace = True)
df['score_3'].fillna(1, inplace = True)
df['score_4'].fillna(1, inplace = True)
df['score_5'].fillna(1, inplace = True)

df.reset_index(inplace=True)
df.drop(['index'],inplace=True,axis=1)

multi_datasets = {}
groups = df.groupby('Essayset')
for name, group in groups:
    multi_datasets[name] = group

pp = []
def different_models(multi_datasets):
    flag = 1
    for i in multi_datasets:
        print(i)
        
        single_df = multi_datasets[i]
        single_df.reset_index(inplace= True)
        single_df.dropna(axis=0,inplace=True)

        test_id = list(single_df.loc[single_df['source_'] ==0]['ID'])
        es = list(single_df.loc[single_df['source_'] ==0]['Essayset'])
        if len(test_id) != len(es):
            print('PANGA')
            break

        single_df['candi_score'] = single_df[['score_1','score_2' ,'score_3' ,'score_4' ,'score_5']].mean(axis=1)
        single_df['candi_score'] = list(map(lambda x : round(x),single_df['candi_score']))
        single_df.drop(['score_1','score_2' ,'score_3' ,'score_4' ,'score_5','ID','index'],inplace=True,axis=1)

        single_df['kitna_aacha']=single_df['candi_score']

        single_df.drop(['min_score','max_score','Essayset','candi_score'],inplace=True,axis=1)

        #all_text  = []
        
        single_df['EssayText']=single_df['EssayText'].apply(preprocess_data)
        #all_text = list(single_df['EssayText'])
        new_l = []
        s = list(single_df['EssayText'])
        for i in s:
            if len(i) < 1:
                new_l.append('ias')
            else:
                new_l.append(i)
        print('Len is')
        print(len(new_l))
        vectors = get_vector(bert_embedding,new_l)
        print('Vectors fetched !!')
        #from sklearn.feature_extraction.text import TfidfVectorizer
        #tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=20, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
        #tfidf.fit(all_text)

        #features = tfidf.transform(single_df['EssayText'])

        new_tfidf_features = pd.DataFrame(vectors)
        new_tfidf_features['clarity'] = list(single_df['clarity'])
        new_tfidf_features['coherent'] = list(single_df['coherent'])
        new_tfidf_features['kitna_aacha'] = list(single_df['kitna_aacha'])
        new_tfidf_features['source_'] = list(single_df['source_'])

        new_tfidf_features.dropna(axis=0,inplace=True)

        all_preprocessed_single_q_data =  pd.get_dummies(new_tfidf_features,columns=['clarity' ,'coherent'])
        preprocessed_main_test = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 0]
        preprocessed_main_train = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 1]

        preprocessed_main_test.drop(['source_','kitna_aacha'],axis =1,inplace=True)

        y_train = (preprocessed_main_train['kitna_aacha'])
        x_train = preprocessed_main_train.drop(['kitna_aacha','source_'],axis =1)

        from sklearn.model_selection import train_test_split
        from sklearn.naive_bayes import MultinomialNB,GaussianNB
        from sklearn.linear_model import LogisticRegression
        X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =0)
    
        num_cla = len(set(y_train))
        print('----')
        print(num_cla)
        
        
        params = {
      'num_leaves': 5,
      'max_bin': 63,
      'num_class':num_cla,
      'min_data_in_leaf': 45,
      'learning_rate': 0.01,
      'min_sum_hessian_in_leaf': 0.000446,
      'bagging_fraction': 0.55,
      'bagging_freq': 5,
      'max_depth': 14,
      'save_binary': True,
      'seed': 31452,
      'feature_fraction_seed': 31415,
      'feature_fraction': 0.51,
      'bagging_seed': 31415,
      'drop_seed': 31415,
      'data_random_seed': 31415,
      'objective': 'multiclass',
      'boosting_type': 'gbdt',
      'verbose': 1,
      'metric': 'multi_logloss',
      'is_unbalance': False,
    #     'categorical_feature': "name:branch_id"
      # 'scale_pos_weight': len(train_df["target"] == 0) / sum(train_df["target"])
    }

        # Import the model we are using
        #from sklearn.ensemble import RandomForestClassifier
        # from tpot import TPOTClassifier
        import lightgbm as lgb

        d_train = lgb.Dataset(X_train, label=y_train_internal)
        # Instantiate model with 1000 decision trees
        #rf = RandomForestClassifier()

        # Train the model on training data
        #rm = rf.fit(X_train, y_train_internal)
        clf = lgb.train(params, d_train,20000)
        
        #pred_internal = rm.predict(preprocessed_main_test)
        #res =list(zip(test_id,es,pred_internal))
        #from sklearn import metrics
        if flag ==0 :
            pred_internal = clf.predict(X_test)
            print(pred_internal)
            pred_internal = list(map(lambda x : max(enumerate(x), key=operator.itemgetter(1))[0],pred_internal))
            from sklearn import metrics
            pp.append(metrics.accuracy_score(y_test_internal, pred_internal))
        if flag ==1:
            pred_internal = clf.predict(preprocessed_main_test)
            pred_internal = list(map(lambda x : max(enumerate(x), key=operator.itemgetter(1))[0],pred_internal))
            res =list(zip(test_id,es,pred_internal))
            pp.append(res)
    return pp
    #res =list(zip(test_id,pred_internal))



/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [52]:
a = different_models(multi_datasets)

1.0
Len is
2195
Vectors fetched !!


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


----
4
2.0
Len is
1679
Vectors fetched !!
----
4
3.0
Len is
2164
Vectors fetched !!
----
3
4.0
Len is
1905
Vectors fetched !!
----
3
5.0
Len is
2343
Vectors fetched !!
----
4
6.0
Len is
2359
Vectors fetched !!
----
4
7.0
Len is
2347
Vectors fetched !!
----
3
8.0
Len is
2350
Vectors fetched !!
----
3
9.0
Len is
2352
Vectors fetched !!
----
3
10.0
Len is
2133
Vectors fetched !!
----
3


In [47]:
len(a)

2195

In [53]:
from itertools import chain
import itertools
l = list(itertools.chain(*a))


In [54]:
res_ = pd.DataFrame(l,columns=['id','essay_set','essay_score'])


In [55]:
res_.to_csv('baseline_lang_model_1.csv',index=False)

In [13]:
df_test.head()

,ID,Essayset,min_score,max_score,clarity,coherent,EssayText,source_
0,1673,1,0,3,average,worst,The procedures I think they should have includ...,0
1,1674,1,0,3,average,worst,"In order to replicate this experiment, you wou...",0
2,1675,1,0,3,above_average,above_average,"In order to replicate their experiment, you wo...",0
3,1676,1,0,3,worst,worst,Pleace a simple of one material into one conta...,0
4,1677,1,0,3,worst,worst,Determin the mass of four different samples ma...,0


In [25]:
import operator
l  = [[9.99690792e-01 ,1, 1.51614726e-07, 4.23026149e-13],[1,2,3,4]]
index, value = max(enumerate(l), key=operator.itemgetter(1))

In [31]:
pred_internal = list(map(lambda x : max(enumerate(x), key=operator.itemgetter(1))[0],l))

In [32]:
pred_internal

[1, 3]

In [10]:
df = pd.read_csv('train_dataset.csv')
list(df['EssayText'])

['Some additional information that we would need to replicate the experiment is how much vinegar should be placed in each identical container, how or what tool to use to measure the mass of the four different samples and how much distilled water to use to rinse the four samples after taking them out of the vinegar.',
 'After reading the expirement, I realized that the additional information you need to replicate the expireiment is one, the amant of vinegar you poured in each container, two, label the containers before you start yar expirement and three, write a conclusion to make sure yar results are accurate.',
 'What you need is more trials, a control set up, and an exact amount of vinegar to pour in each cup/beaker. You could also take and check the mass every 30 min or 1 hour.',
 'The student should list what rock is better and what rock is the worse in the procedure.',
 'For the students to be able to make a replicate, they would need to tell use how much vinegar is used and what 